In [2]:
!pip install moviepy pytube ffmpeg-python asyncffmpeg asynccpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00


In [1]:
from pytube import YouTube
import ffmpeg
import requests
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import json
import os
import warnings
warnings.filterwarnings('ignore')
from moviepy.editor import *
import concurrent.futures
from multiprocessing import process
from threading import Thread
from asynccpu import ProcessTaskPoolExecutor
from asyncffmpeg import FFmpegCoroutineFactory, StreamSpec
import asyncio
import tarfile
from zipfile import ZipFile

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check if mp4 file is playable
def is_mp4_playable(file_path):
  try:
      # Probe the file to get information about it
      probe = ffmpeg.probe(file_path)
      # Check if the file format is recognized as video
      if 'streams' in probe and any(stream['codec_type'] == 'video' for stream in probe['streams']):
          print("MP4 file is playable.")
          return True
      else:
          print("MP4 file is corrupt or non-playable.")
          return False
  except ffmpeg.Error as e:
      print("Error occurred:", e.stderr)
      return False

In [3]:
# Extract audio from given video
def extract_audio(video_url, output_folder):
  if output_folder:
    audio, err = (
      ffmpeg
      .input(video_url)
      .output("pipe:", format='mp3', acodec='libmp3lame', audio_bitrate='320k')
      .run(capture_stdout=True)
    )
    with open(output_folder, 'wb') as f:
        f.write(audio)
    print("Audio extraction complete")
  else:
    print("Audio extraction not done - no output file given")

In [4]:
# Extract frames from given video
def extract_frames(video_url, output_folder, fps=0.1):
  if output_folder:
    (
        ffmpeg
        .input(video_url)
        .filter('fps', fps=fps)
        .output(output_folder + '/frame%d.png', **{'qscale:v': 2})
        .run()
    )
    print("video frames extraction complete")
  else:
    print("Video frames extraction not done - no output file given")

In [5]:
# Process given video/webpage link to extract correct video path and make sure the video is accessible
def process_video(link):

  if len(urlparse(link).netloc) > 0:
    if urlparse(link).netloc == "www.youtube.com":
      yt = YouTube(link)
      video_path = yt.streams[0].url
      return video_path

    elif urlparse(link).netloc == "www.linkedin.com":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_linkedin = json.loads(soup.find('script', type='application/ld+json').text)
      if data_linkedin['isAccessibleForFree'] == True:
        video_path = json.loads(soup.video['data-sources'])[0]['src']
        return video_path
      else:
        print("Sorry! Can't process video. Please make sure the video is free for access.")
        return

    elif urlparse(link).netloc == "www.coursera.org":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_coursera = json.loads(soup.find('script', type='application/ld+json').text)
      video_path = data_coursera['@graph'][1]['contentURL']
      return video_path

    else:
      print("Sorry! Can't process video. Please make sure the video link is valid.")
      return

  else:
    if(is_mp4_playable(link)):
      video_path = link
      return video_path
    else:
      print("Sorry! Can't process video. Please make sure the video file exists and is not corrupted.")
      return

In [6]:
# Parallel process audio and frames extraction
async def main(video_link, output_file_path_audio,output_file_path_frames):

  video_path = process_video(video_link)
  if video_path:
      awaitables = [
            asyncio.to_thread(extract_audio, video_path, output_file_path_audio),
            asyncio.to_thread(extract_frames, video_path, output_file_path_frames)
      ]
      await asyncio.gather(*awaitables)

In [ ]:
# Testing out with an example video
if __name__ == '__main__':
  video_link=input("Video link: ")
  output_file_path_audio=input("MP3 file path to store audio: ")
  output_file_path_frames=input("Folder path to store video frames: ")
  await main(video_link, output_file_path_audio,output_file_path_frames)

https://www.youtube.com/watch?v=Pu5CNKweuHw
/content/drive/My Drive/LLM_Project/youtube_audio_2.mp3
/content/drive/My Drive/LLM_Project/frames
video frames extraction complete
Audio extraction complete


### TVSum Dataset:

In [15]:
# Extracting TVSum data
os.chdir('/content/drive/My Drive/')

with tarfile.open("tvsum50_ver_1_1.tgz", 'r:gz') as tar:
    tar.extractall( path= "/content/drive/My Drive/LLM_Project/tvsum" )

In [17]:
# Extracting videos from TVSum data
os.chdir('/content/drive/MyDrive/LLM_Project/tvsum/ydata-tvsum50-v1_1/')

with ZipFile("ydata-tvsum50-video.zip", 'r') as zObject:
    zObject.extractall( path= "ydata-tvsum50-video" )

In [10]:
# Getting list of videos
video_path = '/content/drive/MyDrive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/video/'
videos=os.listdir(video_path)

In [25]:
# Extracting audio and frames from each video and storing in individual folders (This took around 13 min)
if __name__ == '__main__':
  for video in videos:
    print("\n",video,":")
    output_file_path_frames = '/content/drive/My Drive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/frames/'+video.split('.')[0]
    output_file_path_audio = '/content/drive/My Drive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/audio'
    if not os.path.exists(output_file_path_frames):
      os.makedirs(output_file_path_frames)
    if not os.path.exists(output_file_path_audio):
      os.makedirs(output_file_path_audio)
    await main(video_path+video, output_file_path_audio+'/'+video.split('.')[0]+'.mp3', output_file_path_frames)


 -esJrBWj2d8.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 0tmA_C6XwfM.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 37rzWOQsNIw.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 3eYKfiOEJNs.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 4wU_LUjG5Ic.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 91IHQYk1IQM.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 98MoyGZKHXc.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 _xMr-HKMfVA.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 akI8YFjEmUw.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 AwmHb44_ouw.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction

### SumMe Dataset

In [27]:
# Extracting SumMe data
os.chdir('/content/drive/MyDrive/')

with ZipFile("SumMe.zip", 'r') as zObject:
    zObject.extractall( path= "/content/drive/MyDrive/LLM_Project/SumMe" )

In [7]:
# Getting list of videos
video_path = '/content/drive/MyDrive/LLM_Project/SumMe/SumMe/videos/'
videos=os.listdir(video_path)

In [14]:
# Extracting audio and frames from each video and storing in individual folders (This took around 8 min. Two of the videos were producing error when extracting. Will look into it)
if __name__ == '__main__':
  for video in videos:
    if video.split('.')[1] == "mp4":
      print("\n",video,":")
      output_file_path_frames = '/content/drive/My Drive/LLM_Project/SumMe/SumMe/frames/'+video.split('.')[0]
      output_file_path_audio = '/content/drive/My Drive/LLM_Project/SumMe/SumMe/audio'
      if not os.path.exists(output_file_path_frames):
        os.makedirs(output_file_path_frames)
      if not os.path.exists(output_file_path_audio):
        os.makedirs(output_file_path_audio)
      await main(video_path+video, output_file_path_audio+'/'+video.split('.')[0]+'.mp3', output_file_path_frames)


 Air_Force_One.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Base jumping.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Bearpark_climbing.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Bike Polo.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Bus_in_Rock_Tunnel.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 car_over_camera.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Car_railcrossing.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Cockpit_Landing.mp4 :
MP4 file is playable.
Audio extraction complete
video frames extraction complete

 Cooking.mp4 :
MP4 file is playable.
video frames extraction complete
Audio extraction complete

 Eiffel Tower.mp4 :
MP4 file is playable.
Audio extraction complete